In [95]:
import random
import numpy as np
import pandas as pd
import scipy.stats as sps
import plotly.graph_objs as go

In [96]:
def get_qq_plot(p_values):
    """Рисует распределение p-value"""
    p_values = np.array(p_values)
    probs = []
    x = [0.01 * i for i in range(101)]
    for i in range(101):
        alpha_step = 0.01 * i
        probs.append(p_values[p_values < alpha_step].shape[0] / p_values.shape[0])
    fig = go.Figure([go.Scatter(x=x, y=probs, mode="markers", name="p_value"),
                 go.Scatter(x=x, y=x, mode="lines", name="uniform")])
    fig.update_layout(height=600, width=600, title="Q-Q plot") 
    return fig

In [97]:
def get_power(p_values, alpha=0.05):
    """Оценка мощности критерия, при условии, что значения p_value взяты при наличии 
    различий в сравниваемых выборках 
    """
    p_values = np.array(p_values)
    return p_values[p_values < alpha].shape[0] / p_values.shape[0] * 100

## 1. Длительность теста 
Реализовать формулу подсчета длительности теста, сравнить ее с онлайн калькуляторами (например https://mindbox.ru/tools/ab-test-calculator/ ). При сравнении оценить мощность критерия при указанном изменении и рассчитанном количестве наблюдений в выборке. 

In [175]:
def duration(k, delta_effect, sigma_1, sigma_2, alpha=0.05, beta=0.2):
    z = sps.norm.ppf(1 - alpha/2) + sps.norm.ppf(1-beta)
    n = (k+1) * z ** 2 * (sigma_1 ** 2 + sigma_2 **2 / k) / (delta_effect ** 2)
    return n

Рассчитаем длительность теста для эффекта = 3

In [176]:
effect = 3
alpha = 0.05
sigma_1 = 3
sigma_2 = 3
d = duration(k=1, delta_effect=effect, sigma_1=sigma_1, sigma_2=sigma_2, alpha=alpha, beta=0.2)
d

31.39551893739635

Проверим, что данная длительность действительно обеспечивает обозначенную мощность. Возьмем параметр d и положим в наш эксперимент (округлим в большую сторону)

In [177]:
size = int(d) + 1
n_exp = 1000

p1_values = []
for i in range(n_exp):
    x1_norm = sps.norm.rvs(loc=10, scale=sigma_1, size=size // 2)
    x2_norm = sps.norm.rvs(loc=10 + effect, scale=sigma_2, size=size // 2) # хотим заметить маленький эффект
    p1_value = sps.ttest_ind(x1_norm, x2_norm).pvalue
    p1_values.append(p1_value)
p1_values = np.array(p1_values)

Мощность примерно около 80%, как мы и рассчитывали

In [178]:
get_power(p1_values)

78.2

perfect!

## 2. Метод линеаризации 
Реализовать метод линеаризации. Проверить для него корректность и мощность. Мощность должна быть больше, чем просто на обычных значениях конверсии пользователей.

In [68]:
n_exp = 1000
size = 100
p_values = []
p_values_lin = []
for _ in range(n_exp):
    records = []
    for i in range(size):
        n_views = int(sps.expon.rvs(loc=100, scale=100))
        clicks = sps.bernoulli.rvs(p=0.05, size=n_views)
        records.append([n_views, np.sum(clicks), np.sum(clicks)/ n_views, "A"])
    for i in range(size):
        n_views = int(sps.expon.rvs(loc=100, scale=100))
        clicks = sps.bernoulli.rvs(p=0.05, size=n_views)
        records.append([n_views, np.sum(clicks), np.sum(clicks)/ n_views, "B"])
        
    df_data = pd.DataFrame(records, columns=["views", "clicks", "cr", "group"])
    
    cr_A = df_data[df_data["group"] == "A"]["clicks"].sum() / df_data[df_data["group"] == "A"]["views"].sum()
    df_data["cr_lin"] = df_data["clicks"] - cr_A * df_data["views"]

    x_a = df_data[df_data["group"] == "A"]["cr"]
    x_b = df_data[df_data["group"] == "B"]["cr"]
    
    p_value = sps.ttest_ind(x_a, x_b).pvalue
    p_values.append(p_value)
    
    x_a_lin = df_data[df_data["group"] == "A"]["cr_lin"]
    x_b_lin = df_data[df_data["group"] == "B"]["cr_lin"]
    
    p_value_lin = sps.ttest_ind(x_a_lin, x_b_lin).pvalue
    p_values_lin.append(p_value_lin)
    
p_values = np.array(p_values)
p_values_lin = np.array(p_values_lin)

Проверим корректность критерия для обоих случаев

In [69]:
get_qq_plot(p_values)

In [70]:
get_qq_plot(p_values_lin)

In [72]:
print('Мощность при линеаризации:', get_power(p_values_lin))
print('Обычная мощность:', get_power(p_values))
get_power(p_values_lin) > get_power(p_values)

Мощность при линеаризации: 5.4
Обычная мощность: 5.3


True

Мощность критерия с линеаризацией получилась немного выше, чем на обычных значениях

### 3. CUPED
Реализовать метод CUPED. Проверить для него корректность и мощность. Данные на этапе до A/B тесте необходимо сгенерировать один раз, далее синтетически генерировать только часть, связанную с проведением A/B-теста.

In [91]:
n_exp = 1000
size = 1000
p_values = []
p_values_cuped = []
corr = []
pre_exp = sps.norm.rvs(loc=100, scale=20, size=size)

for i in range(n_exp):
    df_A = pd.DataFrame()
    df_A["user"] = [f"A_{i}" for x in range(size)]
    df_A["pre_exp"] = pre_exp
    df_A["payments"] = sps.expon.rvs(loc=100, scale=100, size=size)
    
    df_B = pd.DataFrame()
    df_B["pre_exp"] = pre_exp
    df_B["user"] = [f"B_{i}" for x in range(size)]
    df_B["payments"] = sps.expon.rvs(loc=100, scale=100, size=size)
    
    p_values.append(sps.ttest_ind(df_A["payments"], df_B["payments"]).pvalue)
    
    x_a = df_A["pre_exp"]
    x_b = df_B["pre_exp"]
    y_a = df_A["payments"]
    y_b = df_B["payments"]
    
    theta = np.cov(x_a, y_a)[0,1] / np.std(x_a)**2
    corr.append(theta)
    
    df_A["payments_cuped"] = df_A["payments"] - theta * df_A["pre_exp"]
    df_B["payments_cuped"] = df_B["payments"] - theta * df_B["pre_exp"]
    
    p_values_cuped.append(sps.ttest_ind(df_A["payments_cuped"], df_B["payments_cuped"]).pvalue)

Проверим корректность критерия 

In [92]:
get_qq_plot(p_values)

In [93]:
# Корректность CUPED
get_qq_plot(p_values_cuped)

Оценим мощность

In [94]:
print('Мощность критерия:', get_power(p_values))
print('Мощность критерия в случае метода CUPED:', get_power(p_values_cuped))

Мощность критерия: 5.0
Мощность критерия в случае метода CUPED: 5.0


В случае наличия эффекта:

In [87]:
n_exp = 1000
size = 1000
p_values = []
p_values_cuped = []
corr = []
pre_exp = sps.norm.rvs(loc=100, scale=20, size=size)

for i in range(n_exp):
    df_A = pd.DataFrame()
    df_A["user"] = [f"A_{i}" for x in range(size)]
    df_A["pre_exp"] = pre_exp
    df_A["payments"] = sps.expon.rvs(loc=100 + 10, scale=100, size=size)
    
    df_B = pd.DataFrame()
    df_B["pre_exp"] = pre_exp
    df_B["user"] = [f"B_{i}" for x in range(size)]
    df_B["payments"] = sps.expon.rvs(loc=100, scale=100, size=size)
    
    p_values.append(sps.ttest_ind(df_A["payments"], df_B["payments"]).pvalue)
    
    x_a = df_A["pre_exp"]
    x_b = df_B["pre_exp"]
    y_a = df_A["payments"]
    y_b = df_B["payments"]
    
    theta = np.cov(x_a, y_a)[0,1] / np.std(x_a)**2
    corr.append(theta)
    
    df_A["payments_cuped"] = df_A["payments"] - theta * df_A["pre_exp"]
    df_B["payments_cuped"] = df_B["payments"] - theta * df_B["pre_exp"]
    
    p_values_cuped.append(sps.ttest_ind(df_A["payments_cuped"], df_B["payments_cuped"]).pvalue)

In [88]:
get_qq_plot(p_values)

In [89]:
# Корректность CUPED
get_qq_plot(p_values_cuped)

In [90]:
print('Мощность критерия:', get_power(p_values))
print('Мощность критерия в случае метода CUPED:', get_power(p_values_cuped))

Мощность критерия: 61.8
Мощность критерия в случае метода CUPED: 61.8
